In [1]:
import re
import math
import warnings
import tqdm
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
from sympy import symbols, Eq, solve
from matplotlib.pyplot import MultipleLocator
from sklearn.preprocessing import MinMaxScaler, Binarizer
from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
warnings.filterwarnings("ignore")


%matplotlib inline
plt.rcParams['font.sans-serif'] = ['SimHei'] 
plt.rcParams['axes.unicode_minus'] = False 

/root/miniconda3/envs/jupyter-env/lib/python3.11/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
data1=pd.read_excel('/root/GDUT/表1-患者列表及临床信息.xlsx')
# data2=pd.read_excel('/root/GDUT/表2-患者影像信息血肿及水肿的体积及位置.xlsx')
# data3=pd.read_excel('/root/GDUT/表3-患者影像信息血肿及水肿的形状及灰度分布.xlsx')
# data4=pd.read_excel('/root/GDUT/表4-答案文件.xlsx')
data5=pd.read_excel('2b聚类数据.xlsx')
data6=pd.read_excel('2a数据_已替换时间戳_已矫正.xlsx')
# data_z=pd.read_excel("治疗方案.xlsx")

# data1.rename(columns={data1.columns[0]: "ID"}, inplace=True)
# data2.rename(columns={data2.columns[0]: "ID"}, inplace=True)

# data_f_1=pd.read_excel('附表1-检索表格-流水号vs时间.xlsx')
# data_f_time=pd.read_excel('时间点.xlsx')

In [3]:
data1[list(data1.columns)[16:]].shape

(160, 7)

In [4]:

data7 = data5

# 转换独热编码
xueya_list = []
for i in data7['血压']:
    y1 = i.split('/')[0]
    y2 = i.split('/')[1]
    if int(y1) > 139 or int(y1) < 90:
        status_i = 0
    elif int(y2) > 89 or int(y2) < 60:
        status_i = 0
    else:
        status_i = 1
    xueya_list.append(status_i)
data7['血压'] = np.array(xueya_list)
data7['性别_男'] = data5['性别'].apply(lambda x: 1 if x == '男' else 0)
data7['性别_女'] = data5['性别'].apply(lambda x: 1 if x == '女' else 0)
x = data7['年龄'].values
scaler = MinMaxScaler()
x_normalized = scaler.fit_transform(x.reshape(-1, 1))
binarizer = Binarizer(threshold=0.5)
x_binarized = binarizer.fit_transform(x_normalized)
data7['年龄'] = x_binarized

In [ ]:
# 归一化
# 比例变量变数值变量，归一化的是数值变量
# 假设data5是你的数据框
hm_columns = data5.filter(like='HM_').columns.tolist()
ed_columns = data5.filter(like='ED_').columns.tolist()
hm_columns.pop(0)
ed_columns.pop(0)
# print(ed_columns)

# 获取'HM_volume'和'ED_volume.0'列的值
hm_volume = data5['HM_volume']
ed_volume = data5['ED_volume']

# 将以'HM_'开头的列与'HM_volume'相乘
data5[hm_columns] = data5[hm_columns].mul(hm_volume, axis=0)

# 将以'ED_'开头的列与'ED_volume.0'相乘
data5[ed_columns] = data5[ed_columns].mul(ed_volume, axis=0)
# 创建 MinMaxScaler 对象
scaler = MinMaxScaler()
# 选择要归一化的列（除了 '年龄' 列之外的所有列）
columns_to_normalize = [col for col in data5.columns if col != '性别']
# 使用 MinMaxScaler 对所选列进行归一化
data5[columns_to_normalize] = scaler.fit_transform(data5[columns_to_normalize])
train = data5[:100]
label = data_kz['是否扩张'].values[:100]
predict = data5[100:]
sm = SMOTE(random_state=0)
xres, yres = sm.fit_resample(train.values, label)

In [5]:
pd.set_option('display.max_rows',None)

#设置列不限制数量
pd.set_option('display.max_columns',None)

#设置value的显示长度为100，默认为50
pd.set_option('max_colwidth',100)

a=list(data7['脑出血前mRS评分'])
data7['脑出血前mRS评分']=[(i-min(a))/(max(a)-min(a)) for i in a]
data7.drop(columns={'性别'},axis=1,inplace=True)

data7 = pd.concat([data7, data1[list(data1.columns)[16:]]], axis=1)
data7.to_excel('2b数据处理结果.xlsx',index=False)
data7.shape

(160, 20)

In [6]:
import pandas as pd
from sklearn.cluster import KMeans, AgglomerativeClustering, Birch, SpectralClustering
#  DBSCAN,
from sklearn.metrics import silhouette_score, calinski_harabasz_score


# 示例数据
data7=pd.read_excel('2b数据处理结果.xlsx')

data = data7[list(data7.columns)[13:]][:100]



In [7]:

# kmeans = KMeans(n_clusters=k, init='k-means++', max_iter=300, random_state=0)
# 定义所需的聚类算法和参数
for c in [3,4,5]:
    print('*'*50,c,'*'*50)
    clustering_algorithms = [
        ("K-Means", KMeans(n_clusters=c)),
        ("Hierarchical Clustering", AgglomerativeClustering(n_clusters=c)),
        ("Birch", Birch(n_clusters=c)),
        ("Spectral Clustering", SpectralClustering(n_clusters=c))
        ]


    # 迭代聚类算法并分别计算评估指标
    for algorithm_name, algorithm in clustering_algorithms:
        algorithm.fit(data)
        labels = algorithm.labels_

        silhouette = silhouette_score(data, labels)
        ch_index = calinski_harabasz_score(data, labels)

        print("Clustering Algorithm:", algorithm_name)
        print("Clustering Labels:", labels)
        print("Silhouette coefficient:", silhouette)
        print("Calinski-Harabasz index:", ch_index)
        print("---------------------------------------")

************************************************** 3 **************************************************
Clustering Algorithm: K-Means
Clustering Labels: [0 2 0 2 2 1 2 2 2 1 1 2 1 2 2 0 1 2 2 2 0 2 2 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 1 0 0 1 1 1 0 0
 1 0 0 1 1 0 0 0 0 0 0 1 0 1 1 0 0 1 1 0 0 1 0 0 0 0]
Silhouette coefficient: 0.6015669590578825
Calinski-Harabasz index: 51.6381976577765
---------------------------------------
Clustering Algorithm: Hierarchical Clustering
Clustering Labels: [2 0 2 0 0 1 0 0 0 1 1 0 1 0 0 2 1 0 0 0 2 0 0 1 2 2 2 2 2 0 2 2 2 2 0 2 2
 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 1 2 2 1 2 0 1 1 1 2 2
 1 2 2 1 1 2 0 2 2 2 2 1 2 1 1 2 0 1 1 2 2 1 2 2 2 2]
Silhouette coefficient: 0.5932275559264848
Calinski-Harabasz index: 46.616926063237365
---------------------------------------
Clustering Algorithm: Birch
Clustering Labels: [1 1 1 2 2 0 1 2 1 0 0 1 0 2 1 1 0 1 2 1 1 1 2 0 1 1 1 1 1 1 1 1 1 1 1 1 1


In [8]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score
from sklearn.model_selection import GridSearchCV
import numpy as np

# 示例数据
data7 = pd.read_excel('2b数据处理结果.xlsx')
data = data7[list(data7.columns)[3:10]]

# 定义所需的聚类算法和参数
clustering_algorithms = [
    ("K-Means", KMeans())  # 不需要指定初始参数
]

# 定义KMeans的超参数搜索范围
param_grid = {
    'n_clusters': [None, 3, 4, 5],  # 尝试不同的聚类数量
    'init': [None, 'k-means++', 'random'],  # 不同的初始化方法
    'max_iter': [None, 1,5,10,25,50, 100, 150, 200, 250],  # 不同的最大迭代次数
    'random_state': [None, 0,1,2], 
}

# 创建GridSearchCV对象
grid_search = GridSearchCV(estimator=KMeans(), param_grid=param_grid, cv=5)

# 执行网格搜索
grid_search.fit(data)

# 输出最佳参数
print("Best Parameters:", grid_search.best_params_)

# 获取最佳参数的KMeans模型
best_kmeans_model = grid_search.best_estimator_

# 使用最佳模型进行聚类
best_kmeans_model.fit(data)
best_labels = best_kmeans_model.labels_

# 计算评估指标
silhouette = silhouette_score(data, best_labels)
ch_index = calinski_harabasz_score(data, best_labels)

print("Clustering Algorithm: K-Means (Tuned)")
print("Clustering Labels:", best_labels)
print("Silhouette coefficient:", silhouette)
print("Calinski-Harabasz index:", ch_index)


Best Parameters: {'init': 'k-means++', 'max_iter': 100, 'n_clusters': 5, 'random_state': None}
Clustering Algorithm: K-Means (Tuned)
Clustering Labels: [3 2 2 4 3 2 2 2 4 2 4 2 4 3 0 0 4 3 2 4 2 2 1 1 2 2 0 2 2 0 2 1 2 2 0 2 2
 2 2 2 2 2 0 2 2 3 2 2 2 3 3 2 0 2 2 0 3 0 4 4 0 0 2 2 3 2 4 2 4 4 2 2 2 2
 2 0 2 1 4 2 3 1 0 1 2 2 1 2 2 2 2 0 1 1 2 3 2 0 2 1 0 4 4 1 4 0 2 2 3 3 0
 2 2 2 1 4 4 2 1 2 2 2 4 2 2 3 3 2 2 4 2 4 2 2 4 0 2 2 4 0 2 2 2 1 4 1 2 1
 4 4 4 2 2 2 2 2 4 2 1 0]
Silhouette coefficient: 0.5772323973789996
Calinski-Harabasz index: 62.71049428576463


In [33]:
# import json

# # 0.693
# # 创建一个包含数组的字典
# data = {"best_labels": best_labels.tolist()}

# # 保存为JSON文件
# with open("best_labels.json", "w") as json_file:
#     json.dump(data, json_file)

# # 要在其他Python脚本中读取这个JSON文件，你可以使用json.load()函数。

In [34]:
# data